In [1]:
import time
from datetime import datetime, timezone
from dateutil import parser

import requests

In [2]:
parameters = notebookutils.variableLibrary.getLibrary("VL_parameters")
semantic_model_id = parameters.semantic_model_id
sql_endpoint_id = parameters.sql_endpoint_id
headers = {"Authorization": f"Bearer {notebookutils.credentials.getToken('pbi')}"}

## force refresh of SQL analytics endpoint

In [28]:
workspace_id = notebookutils.runtime.context["currentWorkspaceId"]
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/sqlEndpoints/{sql_endpoint_id}/refreshMetadata"
response = requests.post(url, headers=headers, json={})
response.raise_for_status()

if response.status_code == 200:
    print(response.json())

{'value': [{'tableName': 'dbo.acr', 'status': 'NotRun', 'startDateTime': '2025-09-09T08:53:11.6573044Z', 'endDateTime': '2025-09-09T08:53:11.7354364Z', 'lastSuccessfulSyncDateTime': '2025-09-09T05:35:35.9684726Z'}]}


## wait to ensure refresh is completed

In [33]:
if response.status_code == 202:
    end_datetime_str = response.json()['value'][0]['endDateTime']

    # Parse the endDateTime to a datetime object (UTC)
    end_datetime = parser.isoparse(end_datetime_str)

    # Get current UTC time
    now = datetime.now(timezone.utc)

    # Calculate how many seconds to wait
    wait_seconds = (end_datetime - now).total_seconds()

    # If wait_seconds is positive, sleep; otherwise, continue immediately
    if wait_seconds > 0:
        print(f"Waiting for {wait_seconds:.2f} seconds until {end_datetime} before next request...")
        time.sleep(wait_seconds)
    else:
        print("endDateTime is in the past.")
        
print("Let's wait an additional 30 seconds to be sure.")        
time.sleep(30)

endDateTime is in the past. Proceeding immediately.


## Refresh semantic model (which is based on SQL AE)

In [16]:
headers = {"Authorization": f"Bearer {notebookutils.credentials.getToken('pbi')}"}
response = requests.post(f"https://api.powerbi.com/v1.0/myorg/datasets/{semantic_model_id}/refreshes", headers=headers)
response.raise_for_status()
response

<Response [202]>

In [25]:
if response.status_code == 202:
    refresh_status_url = f"https://api.powerbi.com/v1.0/myorg/datasets/{semantic_model_id}/refreshes"
    status_response = requests.get(refresh_status_url, headers=headers)
    refreshes = status_response.json()

    # Look for your specific refresh using the RequestId
    for refresh in refreshes['value']:
        if refresh.get('requestId') == response.headers["RequestId"]:
            print(f"Status: {refresh['status']}")
            print(f"Start Time: {refresh.get('startTime', 'N/A')}")
            print(f"End Time: {refresh.get('endTime', 'N/A')}")
            break
else:
    print(response.json())

Status: Completed
Start Time: 2025-09-09T08:33:36.587Z
End Time: 2025-09-09T08:33:53.467Z
